In [1]:
from eins import EinsOp, Reductions as Red

import jax
import jax.numpy as jnp
import numpy as np

In [2]:
x = jnp.array(np.random.randn(1024, 256, 3))
y = jnp.array(np.random.randn(1024, 256, 3))

z4 = EinsOp('b n1 d, b n2 d -> b n1 n2', combine='add', reduce=Red.l2_norm)(x, -y)

# Version without eins. Note how easy it would be to write x[:, None, ...] - y[:, :, None, ...],
# which would lead to the transposed version of the pairwise distances you want.
z5 = jnp.sqrt(jnp.sum(jnp.square(x[:, :, None, ...] - y[:, None, ...]), axis=-1))

jnp.max(jnp.abs(z4 - z5))

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
/home/nicholas/anaconda3/lib/python3.11/site-packages/jax/_src/numpy/array_methods.py:64: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return lax_numpy.astype(arr, dtype)


Array(9.536743e-07, dtype=float32)

In [3]:
@jax.jit
def jnp_pairwise_dist(x, y):
    return jnp.sqrt(jnp.sum(jnp.square(x[:, :, None, ...] - y[:, None, ...]), axis=-1))

@jax.jit
def ein_pairwise_dist(x, y):
    return EinsOp('b n1 d, b n2 d -> b n1 n2', combine='add', reduce=Red.l2_norm)(x, -y)

d1 = jnp_pairwise_dist(x, y)
d2 = ein_pairwise_dist(x, y)

In [4]:
%%timeit
jnp_pairwise_dist(x, y).block_until_ready()

22.3 ms ± 1.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit
ein_pairwise_dist(x, y).block_until_ready()

250 ms ± 16.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
jax.make_jaxpr(jnp_pairwise_dist)(x, y)

{ lambda ; a:f32[1024,256,3] b:f32[1024,256,3]. let
    c:f32[1024,256,256] = pjit[
      jaxpr={ lambda ; d:f32[1024,256,3] e:f32[1024,256,3]. let
          f:f32[1024,256,1,3] = broadcast_in_dim[
            broadcast_dimensions=(0, 1, 3)
            shape=(1024, 256, 1, 3)
          ] d
          g:f32[1024,1,256,3] = broadcast_in_dim[
            broadcast_dimensions=(0, 2, 3)
            shape=(1024, 1, 256, 3)
          ] e
          h:f32[1024,256,256,3] = sub f g
          i:f32[1024,256,256,3] = integer_pow[y=2] h
          j:f32[1024,256,256] = reduce_sum[axes=(3,)] i
          k:f32[1024,256,256] = sqrt j
        in (k,) }
      name=jnp_pairwise_dist
    ] a b
  in (c,) }

In [7]:
jax.make_jaxpr(ein_pairwise_dist)(x, y)

{ lambda ; a:f32[1024,256,3] b:f32[1024,256,3]. let
    c:f32[1024,256,256] = pjit[
      jaxpr={ lambda ; d:f32[1024,256,3] e:f32[1024,256,3]. let
          f:f32[1024,256,3] = neg e
          g:f32[1024,256,3,1] = broadcast_in_dim[
            broadcast_dimensions=(0, 1, 2)
            shape=(1024, 256, 3, 1)
          ] d
          h:f32[1024,3,256] = transpose[permutation=(0, 2, 1)] f
          i:f32[1024,1,3,256] = broadcast_in_dim[
            broadcast_dimensions=(0, 2, 3)
            shape=(1024, 1, 3, 256)
          ] h
          j:f32[1024,256,3,256] = add g i
          k:f32[1024,256,3,256] = integer_pow[y=2] j
          l:f32[1024,256,256] = reduce_sum[axes=(2,)] k
          m:f32[1024,256,256] = sqrt l
        in (m,) }
      name=ein_pairwise_dist
    ] a b
  in (c,) }